**🧠 Voice-Based Cognitive Decline Detection Pipeline**

**STEP 1: Transcribe Audio to Text using Whisper**

In [24]:
import whisper
import os
from pathlib import Path
import zipfile

In [25]:
model = whisper.load_model("base")  # Or 'small', 'medium', etc.

def transcribe_audio(file_path):
    result = model.transcribe(file_path)
    return result['text'], result['segments']  # Segments include timestamps

**STEP 2: Extract ZIP of audio files**

In [26]:
def extract_zip(zip_path, extract_to="extracted_audio"):
    # Check if the ZIP file exists
    if not os.path.exists(zip_path):
        raise ValueError(f"❌ The ZIP file does not exist at the specified path: {zip_path}")
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # List all files inside the ZIP archive
        zip_file_list = zip_ref.namelist()
        print(f"📂 Files in ZIP: {zip_file_list}")
        
        zip_ref.extractall(extract_to)
        print(f"✅ Extracted files to: {extract_to}")

    # Get all .wav files, including those in subdirectories
    extracted_files = []
    for root, dirs, files in os.walk(extract_to):
        for file in files:
            if file.endswith('.wav'):
                extracted_files.append(str(Path(root) / file))

    print(f"🎧 Extracted .wav files: {extracted_files}")

    return extracted_files

**STEP 3: Extract NLP Features**

In [27]:
import re
import numpy as np

In [28]:
def extract_nlp_features(text):
    hesitation_words = ["uh", "um", "erm"]
    words = text.lower().split()
    
    num_words = len(words)
    num_hesitations = sum(words.count(h) for h in hesitation_words)
    hesitation_ratio = num_hesitations / num_words if num_words > 0 else 0

    avg_sentence_length = np.mean([len(s.split()) for s in re.split(r'[.!?]', text) if len(s.strip()) > 0])

    return {
        "num_words": num_words,
        "num_hesitations": num_hesitations,
        "hesitation_ratio": hesitation_ratio,
        "avg_sentence_length": avg_sentence_length,
    }

**STEP 4: Extract Audio Features**

In [29]:
import librosa

In [30]:
def extract_audio_features(file_path):
    y, sr = librosa.load(file_path)
    duration = librosa.get_duration(y=y, sr=sr)
    
    pitch = librosa.yin(y, fmin=50, fmax=300)
    pitch_std = np.std(pitch)
    
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

    return {
        "duration_sec": duration,
        "pitch_std": pitch_std,
        "tempo_bpm": tempo,
    }

**STEP 5: Combine + Apply ML (Unsupervised)**

In [31]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
def analyze_files(audio_files):
    rows = []
    
    # Ensure audio_files list is populated
    if not audio_files:
        raise ValueError("❌ No audio files found to process!")

    for file_path in audio_files:
        try:
            # Log file being processed
            print(f"📁 Processing file: {file_path}")

            # Check if file exists
            if not Path(file_path).exists():
                print(f"⚠️ File does not exist: {file_path}")
                continue

            # Transcribe audio to text
            transcript, _ = transcribe_audio(file_path)
            print(f"🔊 Transcript for {file_path}: {transcript[:100]}...")  # Print first 100 chars of transcript
            
            if not transcript:
                print(f"⚠️ Empty transcript for {file_path}, skipping.")
                continue

            # Extract NLP features
            nlp_feats = extract_nlp_features(transcript)
            print(f"📝 NLP features for {file_path}: {nlp_feats}")  # Print NLP features
            if not nlp_feats:
                print(f"⚠️ Empty NLP features for {file_path}, skipping.")
                continue

            # Extract audio features
            audio_feats = extract_audio_features(file_path)
            print(f"🎧 Audio features for {file_path}: {audio_feats}")  # Print audio features
            if not audio_feats:
                print(f"⚠️ Empty audio features for {file_path}, skipping.")
                continue

            # Combine all features into one dictionary
            all_feats = {**nlp_feats, **audio_feats, "file": Path(file_path).name}
            print(f"✅ Combined features for {file_path}: {all_feats}")  # Print combined features
            
            rows.append(all_feats)

        except Exception as e:
            print(f"⚠️ Error processing {file_path}: {e}")
    
    # Create DataFrame from collected rows
    df = pd.DataFrame(rows)
    
    # Print DataFrame shape and column types for debugging
    print("\nDataFrame created:")
    print("DataFrame shape:", df.shape)
    print("DataFrame columns:", df.columns)
    print("DataFrame types:", df.dtypes)

    # Check for numeric columns for scaling
    numeric_df = df.select_dtypes(include=[np.number])
    print("\nNumeric features found for scaling:")
    print(numeric_df.head())

    if numeric_df.empty:
        raise ValueError("❌ No numeric features found for scaling!")
    
    # ML: Anomaly detection using Isolation Forest
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(numeric_df)
    model = IsolationForest(contamination=0.2)
    df['risk_flag'] = model.fit_predict(X_scaled)  # -1 indicates anomaly, 1 indicates normal

    return df

**STEP 6: Visualize Results**

In [33]:
def plot_features(df):
    sns.pairplot(df, hue='risk_flag', diag_kind='kde')
    plt.suptitle("Cognitive Decline Feature Trends", y=1.02)
    plt.show()

**API-ready Prediction Function**

In [34]:
def predict_risk_from_audio(file_path, scaler, model):
    transcript, _ = transcribe_audio(file_path)
    nlp_feats = extract_nlp_features(transcript)
    audio_feats = extract_audio_features(file_path)
    all_feats = {**nlp_feats, **audio_feats}
    df = pd.DataFrame([all_feats])
    X_scaled = scaler.transform(df)
    return model.predict(X_scaled)[0]  # -1 = at risk

In [35]:
zip_path = "/Users/surabhigautam/Documents/Python/Datasets/speech_dataset.zip"

In [36]:
## Step 1: Extract WAV files from ZIP

In [37]:
audio_files = extract_zip(zip_path)

📂 Files in ZIP: ['Actor_01/03-01-01-01-01-01-01.wav', 'Actor_01/03-01-01-01-01-02-01.wav', 'Actor_01/03-01-01-01-02-01-01.wav', 'Actor_01/03-01-01-01-02-02-01.wav', 'Actor_01/03-01-02-01-01-01-01.wav', 'Actor_01/03-01-02-01-01-02-01.wav', 'Actor_01/03-01-02-01-02-01-01.wav', 'Actor_01/03-01-02-01-02-02-01.wav', 'Actor_01/03-01-02-02-01-01-01.wav', 'Actor_01/03-01-02-02-01-02-01.wav', 'Actor_01/03-01-02-02-02-01-01.wav', 'Actor_01/03-01-02-02-02-02-01.wav', 'Actor_01/03-01-03-01-01-01-01.wav', 'Actor_01/03-01-03-01-01-02-01.wav', 'Actor_01/03-01-03-01-02-01-01.wav', 'Actor_01/03-01-03-01-02-02-01.wav', 'Actor_01/03-01-03-02-01-01-01.wav', 'Actor_01/03-01-03-02-01-02-01.wav', 'Actor_01/03-01-03-02-02-01-01.wav', 'Actor_01/03-01-03-02-02-02-01.wav', 'Actor_01/03-01-04-01-01-01-01.wav', 'Actor_01/03-01-04-01-01-02-01.wav', 'Actor_01/03-01-04-01-02-01-01.wav', 'Actor_01/03-01-04-01-02-02-01.wav', 'Actor_01/03-01-04-02-01-01-01.wav', 'Actor_01/03-01-04-02-01-02-01.wav', 'Actor_01/03-01-04-02

In [38]:
# Step 2: Analyze audio + text features

In [ ]:
df_results = analyze_files(audio_files)

📁 Processing file: extracted_audio/Actor_16/03-01-05-01-02-01-16.wav


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🔊 Transcript for extracted_audio/Actor_16/03-01-05-01-02-01-16.wav:  Dogs are sitting by the door....
📝 NLP features for extracted_audio/Actor_16/03-01-05-01-02-01-16.wav: {'num_words': 6, 'num_hesitations': 0, 'hesitation_ratio': 0.0, 'avg_sentence_length': np.float64(6.0)}
🎧 Audio features for extracted_audio/Actor_16/03-01-05-01-02-01-16.wav: {'duration_sec': 3.9039002267573695, 'pitch_std': np.float64(83.92316810965904), 'tempo_bpm': array([123.046875])}
✅ Combined features for extracted_audio/Actor_16/03-01-05-01-02-01-16.wav: {'num_words': 6, 'num_hesitations': 0, 'hesitation_ratio': 0.0, 'avg_sentence_length': np.float64(6.0), 'duration_sec': 3.9039002267573695, 'pitch_std': np.float64(83.92316810965904), 'tempo_bpm': array([123.046875]), 'file': '03-01-05-01-02-01-16.wav'}
📁 Processing file: extracted_audio/Actor_16/03-01-06-01-02-02-16.wav


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🔊 Transcript for extracted_audio/Actor_16/03-01-06-01-02-02-16.wav:  Dogs are sitting by the door....
📝 NLP features for extracted_audio/Actor_16/03-01-06-01-02-02-16.wav: {'num_words': 6, 'num_hesitations': 0, 'hesitation_ratio': 0.0, 'avg_sentence_length': np.float64(6.0)}
🎧 Audio features for extracted_audio/Actor_16/03-01-06-01-02-02-16.wav: {'duration_sec': 3.570249433106576, 'pitch_std': np.float64(96.5707366462613), 'tempo_bpm': array([103.359375])}
✅ Combined features for extracted_audio/Actor_16/03-01-06-01-02-02-16.wav: {'num_words': 6, 'num_hesitations': 0, 'hesitation_ratio': 0.0, 'avg_sentence_length': np.float64(6.0), 'duration_sec': 3.570249433106576, 'pitch_std': np.float64(96.5707366462613), 'tempo_bpm': array([103.359375]), 'file': '03-01-06-01-02-02-16.wav'}
📁 Processing file: extracted_audio/Actor_16/03-01-06-02-01-02-16.wav


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🔊 Transcript for extracted_audio/Actor_19/03-01-01-01-01-01-19.wav:  Kids are talking by the door....
📝 NLP features for extracted_audio/Actor_19/03-01-01-01-01-01-19.wav: {'num_words': 6, 'num_hesitations': 0, 'hesitation_ratio': 0.0, 'avg_sentence_length': np.float64(6.0)}
🎧 Audio features for extracted_audio/Actor_19/03-01-01-01-01-01-19.wav: {'duration_sec': 3.3700680272108845, 'pitch_std': np.float64(56.824002968885445), 'tempo_bpm': array([143.5546875])}
✅ Combined features for extracted_audio/Actor_19/03-01-01-01-01-01-19.wav: {'num_words': 6, 'num_hesitations': 0, 'hesitation_ratio': 0.0, 'avg_sentence_length': np.float64(6.0), 'duration_sec': 3.3700680272108845, 'pitch_std': np.float64(56.824002968885445), 'tempo_bpm': array([143.5546875]), 'file': '03-01-01-01-01-01-19.wav'}
📁 Processing file: extracted_audio/Actor_19/03-01-03-02-02-02-19.wav


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🔊 Transcript for extracted_audio/Actor_19/03-01-03-02-02-02-19.wav:  Dogs are sitting by the door....
📝 NLP features for extracted_audio/Actor_19/03-01-03-02-02-02-19.wav: {'num_words': 6, 'num_hesitations': 0, 'hesitation_ratio': 0.0, 'avg_sentence_length': np.float64(6.0)}
🎧 Audio features for extracted_audio/Actor_19/03-01-03-02-02-02-19.wav: {'duration_sec': 4.004036281179138, 'pitch_std': np.float64(74.58761878246577), 'tempo_bpm': array([184.5703125])}
✅ Combined features for extracted_audio/Actor_19/03-01-03-02-02-02-19.wav: {'num_words': 6, 'num_hesitations': 0, 'hesitation_ratio': 0.0, 'avg_sentence_length': np.float64(6.0), 'duration_sec': 4.004036281179138, 'pitch_std': np.float64(74.58761878246577), 'tempo_bpm': array([184.5703125]), 'file': '03-01-03-02-02-02-19.wav'}
📁 Processing file: extracted_audio/Actor_19/03-01-03-01-01-02-19.wav


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
# Step 3: Plot feature trends and risk anomalies

In [ ]:
plot_features(df_results)

In [ ]:
# Step 4: Export results to CSV

In [ ]:
output_csv = "cognitive_risk_report.csv"
df_results.to_csv(output_csv, index=False)
print(f"📁 Results saved to: {output_csv}")